# Julius Baer - Part 1: Real-Time AML Monitoring System

## System Overview

This notebook implements a comprehensive Anti-Money Laundering (AML) monitoring system for real-time transaction analysis and alert generation.

### Key Features:
- Real-time transaction risk scoring
- KYC/EDD compliance monitoring
- Multi-layered alert system for different teams
- Comprehensive audit trail generation
- Regulatory compliance tracking

### Compliance Standards:
- FINMA AML regulations
- HKMA AML guidelines
- FATF recommendations
- Basel Committee standards

### Target Users:
- **Front Office**: Relationship managers and client-facing teams
- **Compliance Teams**: Risk officers and compliance analysts
- **Legal Teams**: Legal counsel and regulatory affairs

## 1. System Setup and Imports

Installing required libraries and setting up the transaction analysis environment.

In [ ]:
# Core Libraries
import pandas as pd
import numpy as np
from datetime import datetime, timedelta
from typing import Dict, List, Tuple, Optional
import logging
from dataclasses import dataclass
from enum import Enum
import warnings
warnings.filterwarnings('ignore')

# Data Visualization
import matplotlib.pyplot as plt
import seaborn as sns
import plotly.express as px
import plotly.graph_objects as go
from plotly.subplots import make_subplots

# System Configuration
RISK_WEIGHTS = {
    "kyc_overdue": 20,
    "edd_gap": 30,
    "sow_missing": 15,
    "kyc_recency_over_365d": 10,
    "kyc_overdue_over_180d": 10,
    "large_transaction": 25,
    "high_risk_country": 20,
    "round_amount": 5,
    "rapid_succession": 15,
    "cash_intensive": 10
}

print("AML Monitoring System initialized successfully")
print(f"Risk weights configured for {len(RISK_WEIGHTS)} factors")

In [ ]:
# AML System Classes and Enums

class RiskLevel(Enum):
    LOW = "Low"
    MEDIUM = "Medium"
    HIGH = "High"
    CRITICAL = "Critical"

class AlertType(Enum):
    LARGE_TRANSACTION = "Large Transaction"
    UNUSUAL_PATTERN = "Unusual Pattern"
    HIGH_RISK_COUNTRY = "High Risk Country"
    ROUND_AMOUNT = "Round Amount"
    RAPID_SUCCESSION = "Rapid Succession"
    SANCTIONS_HIT = "Sanctions Hit"
    PEP_INVOLVEMENT = "PEP Involvement"
    CASH_INTENSIVE = "Cash Intensive"
    STRUCTURING = "Structuring"
    KYC_OVERDUE = "KYC Overdue"
    EDD_GAP = "EDD Gap"
    SOW_MISSING = "SOW Missing"

@dataclass
class AMLAlert:
    alert_id: str
    transaction_id: str
    alert_type: AlertType
    risk_level: RiskLevel
    description: str
    risk_score: float
    timestamp: datetime
    customer_id: str
    amount: float
    currency: str
    triggered_rules: List[str]
    requires_action: bool
    target_team: str  # Front, Compliance, or Legal

print("AML classes and data structures defined")
print(f"Alert types configured: {len(AlertType)}")
print(f"Risk levels available: {len(RiskLevel)}")

## 2. Data Loading and Preprocessing

Loading transaction data and performing initial validation and preprocessing.

In [ ]:
# Load Transaction Data
try:
    df = pd.read_csv("transactions_mock_1000_for_participants.csv")
    print(f"Successfully loaded {len(df):,} transactions")
except FileNotFoundError:
    print("Transaction file not found. Creating sample data...")
    # Create sample data if file doesn't exist
    np.random.seed(42)
    n_transactions = 1000
    
    df = pd.DataFrame({
        'transaction_id': [f'TXN_{i:06d}' for i in range(1, n_transactions + 1)],
        'customer_id': [f'CUST_{np.random.randint(1000, 9999):04d}' for _ in range(n_transactions)],
        'amount': np.random.lognormal(mean=10, sigma=1.5, size=n_transactions),
        'currency': np.random.choice(['USD', 'EUR', 'CHF', 'GBP', 'SGD'], n_transactions),
        'booking_datetime': [datetime.now() - timedelta(days=np.random.randint(0, 365)) for _ in range(n_transactions)],
        'originator_country': np.random.choice(['US', 'CH', 'UK', 'DE', 'FR', 'SG', 'HK', 'CN', 'RU', 'IR'], n_transactions),
        'beneficiary_country': np.random.choice(['US', 'CH', 'UK', 'DE', 'FR', 'SG', 'HK', 'CN', 'RU', 'IR'], n_transactions),
        'customer_risk_rating': np.random.choice(['Low', 'Medium', 'High'], n_transactions, p=[0.7, 0.25, 0.05]),
        'is_cash_transaction': np.random.choice([True, False], n_transactions, p=[0.1, 0.9]),
        'booking_jurisdiction': np.random.choice(['US', 'CH', 'UK', 'SG', 'HK'], n_transactions)
    })

# Data Type Conversions
df['booking_datetime'] = pd.to_datetime(df['booking_datetime'])
df['amount'] = pd.to_numeric(df['amount'], errors='coerce')

print(f"Dataset shape: {df.shape}")
print(f"Date range: {df['booking_datetime'].min()} to {df['booking_datetime'].max()}")
print(f"Amount range: ${df['amount'].min():,.2f} to ${df['amount'].max():,.2f}")

# Display sample data
print("\nSample transactions:")
display(df.head())

## 3. Transaction Analysis Engine

Core risk scoring engine for analyzing transactions against AML rules and regulations.

In [ ]:
class TransactionAnalysisEngine:
    """Core engine for analyzing transactions against AML rules"""
    
    def __init__(self):
        self.high_risk_countries = ['IR', 'KP', 'SY', 'AF', 'MM', 'BY', 'RU', 'CN']
        self.large_amount_threshold = 1000000  # 1M threshold
        self.cash_threshold = 10000
        self.rapid_succession_window = 3600  # 1 hour
        self.round_amount_threshold = 0.001  # For detecting round amounts
        
    def calculate_risk_score(self, transaction: pd.Series) -> Tuple[float, List[str]]:
        """Calculate composite risk score and triggered rules"""
        risk_score = 0.0
        triggered_rules = []
        
        # Amount-based risk
        if transaction['amount'] > self.large_amount_threshold:
            risk_score += RISK_WEIGHTS['large_transaction']
            triggered_rules.append(f"Large transaction (${transaction['amount']:,.2f})")
        
        # Country risk
        if transaction['originator_country'] in self.high_risk_countries:
            risk_score += RISK_WEIGHTS['high_risk_country']
            triggered_rules.append(f"High-risk originator country ({transaction['originator_country']})")
            
        if transaction['beneficiary_country'] in self.high_risk_countries:
            risk_score += RISK_WEIGHTS['high_risk_country']
            triggered_rules.append(f"High-risk beneficiary country ({transaction['beneficiary_country']})")
        
        # Customer risk
        if transaction['customer_risk_rating'] == 'High':
            risk_score += 20
            triggered_rules.append("High-risk customer")
        elif transaction['customer_risk_rating'] == 'Medium':
            risk_score += 10
            triggered_rules.append("Medium-risk customer")
        
        # Cash transaction risk
        if transaction.get('is_cash_transaction', False) and transaction['amount'] > self.cash_threshold:
            risk_score += RISK_WEIGHTS['cash_intensive']
            triggered_rules.append(f"Large cash transaction (${transaction['amount']:,.2f})")
        
        # Round amount detection (potential structuring)
        if abs(transaction['amount'] % 1000) < self.round_amount_threshold:
            risk_score += RISK_WEIGHTS['round_amount']
            triggered_rules.append("Round amount (potential structuring)")
        
        return risk_score, triggered_rules
    
    def determine_risk_level(self, risk_score: float) -> RiskLevel:
        """Determine risk level based on composite score"""
        if risk_score >= 70:
            return RiskLevel.CRITICAL
        elif risk_score >= 50:
            return RiskLevel.HIGH
        elif risk_score >= 25:
            return RiskLevel.MEDIUM
        else:
            return RiskLevel.LOW
    
    def determine_target_team(self, risk_level: RiskLevel, alert_types: List[str]) -> str:
        """Determine which team should handle the alert"""
        if risk_level in [RiskLevel.CRITICAL, RiskLevel.HIGH]:
            return "Legal"
        elif risk_level == RiskLevel.MEDIUM:
            return "Compliance"
        else:
            return "Front Office"

# Initialize the analysis engine
analysis_engine = TransactionAnalysisEngine()
print("Transaction Analysis Engine initialized")
print(f"High-risk countries: {len(analysis_engine.high_risk_countries)}")
print(f"Large amount threshold: ${analysis_engine.large_amount_threshold:,}")

## 4. KYC/EDD Compliance Monitoring

Implementing Know Your Customer (KYC) and Enhanced Due Diligence (EDD) monitoring for regulatory compliance.

In [ ]:
# KYC/EDD Feature Engineering

# Create mock KYC data if not available
if 'kyc_due_date' not in df.columns:
    np.random.seed(43)
    # Mock KYC data generation
    df['kyc_last_completed'] = df['booking_datetime'] - pd.to_timedelta(np.random.randint(1, 730, len(df)), unit='D')
    df['kyc_due_date'] = df['kyc_last_completed'] + pd.to_timedelta(365, unit='D')
    df['edd_required'] = np.random.choice([True, False], len(df), p=[0.3, 0.7])
    df['edd_performed'] = np.where(df['edd_required'], 
                                  np.random.choice([True, False], len(df), p=[0.8, 0.2]), 
                                  True)
    df['sow_documented'] = np.random.choice([True, False], len(df), p=[0.9, 0.1])

# Calculate KYC/EDD compliance features
df['kyc_overdue'] = (df['booking_datetime'] > df['kyc_due_date']).astype(int)
df['kyc_days_overdue'] = (df['booking_datetime'] - df['kyc_due_date']).dt.days.clip(lower=0)
df['kyc_recency_days'] = (df['booking_datetime'] - df['kyc_last_completed']).dt.days
df['edd_gap'] = ((df['edd_required']) & (~df['edd_performed'])).astype(int)
df['sow_missing'] = (~df['sow_documented']).astype(int)

# Derived flags
df['kyc_recency_over_365d'] = (df['kyc_recency_days'] > 365).astype(int)
df['kyc_overdue_over_180d'] = (df['kyc_days_overdue'] > 180).astype(int)

def calculate_kyc_edd_risk(row):
    """Calculate KYC/EDD specific risk score"""
    score = 0
    score += RISK_WEIGHTS['kyc_overdue'] * row['kyc_overdue']
    score += RISK_WEIGHTS['edd_gap'] * row['edd_gap']
    score += RISK_WEIGHTS['sow_missing'] * row['sow_missing']
    score += RISK_WEIGHTS['kyc_recency_over_365d'] * row['kyc_recency_over_365d']
    score += RISK_WEIGHTS['kyc_overdue_over_180d'] * row['kyc_overdue_over_180d']
    return score

df['kyc_edd_risk_score'] = df.apply(calculate_kyc_edd_risk, axis=1)

# Generate KYC/EDD alert reasons
def get_kyc_alert_reason(row):
    alerts = []
    if row['kyc_overdue']: alerts.append("KYC overdue")
    if row['edd_gap']: alerts.append("EDD required but not performed")
    if row['sow_missing']: alerts.append("Missing Source of Wealth documentation")
    if row['kyc_overdue_over_180d']: alerts.append("KYC overdue > 6 months")
    if row['kyc_recency_over_365d']: alerts.append("KYC not updated > 1 year")
    return "; ".join(alerts) if alerts else "No KYC/EDD issues"

df['kyc_edd_alert_reason'] = df.apply(get_kyc_alert_reason, axis=1)

print("KYC/EDD compliance monitoring setup complete")
print(f"KYC overdue transactions: {df['kyc_overdue'].sum():,}")
print(f"EDD gaps identified: {df['edd_gap'].sum():,}")
print(f"Missing SOW documentation: {df['sow_missing'].sum():,}")
print(f"KYC recency issues (>365 days): {df['kyc_recency_over_365d'].sum():,}")

## 5. Comprehensive Risk Analysis

Performing complete transaction risk analysis combining multiple risk factors.

In [ ]:
# Comprehensive Transaction Risk Analysis

# Calculate transaction-specific risk scores
transaction_risk_scores = []
transaction_triggered_rules = []

for idx, transaction in df.iterrows():
    risk_score, triggered_rules = analysis_engine.calculate_risk_score(transaction)
    transaction_risk_scores.append(risk_score)
    transaction_triggered_rules.append(triggered_rules)

df['transaction_risk_score'] = transaction_risk_scores
df['triggered_rules'] = transaction_triggered_rules

# Calculate total composite risk score
df['total_risk_score'] = df['kyc_edd_risk_score'] + df['transaction_risk_score']

# Determine risk levels
df['risk_level'] = df['total_risk_score'].apply(analysis_engine.determine_risk_level)
df['risk_level_str'] = df['risk_level'].apply(lambda x: x.value)

# Determine target teams
df['target_team'] = df.apply(lambda row: analysis_engine.determine_target_team(
    row['risk_level'], row['triggered_rules']), axis=1)

# Add cross-border transaction analysis
df['is_cross_border'] = (df['originator_country'] != df['beneficiary_country']).astype(int)

# Calculate additional metrics
df['risk_category'] = pd.cut(df['total_risk_score'], 
                           bins=[-1, 25, 50, 75, 150], 
                           labels=['Low', 'Medium', 'High', 'Critical'])

print("Comprehensive risk analysis completed")
print(f"Total transactions analyzed: {len(df):,}")

# Risk distribution summary
risk_summary = df['risk_level_str'].value_counts()
print("\nRisk Level Distribution:")
for level, count in risk_summary.items():
    percentage = (count / len(df)) * 100
    print(f"  {level}: {count:,} transactions ({percentage:.1f}%)")

# Team assignment summary
team_summary = df['target_team'].value_counts()
print("\nTeam Assignment Summary:")
for team, count in team_summary.items():
    percentage = (count / len(df)) * 100
    print(f"  {team}: {count:,} transactions ({percentage:.1f}%)")

print(f"\nRisk Score Statistics:")
print(f"  Mean: {df['total_risk_score'].mean():.2f}")
print(f"  Median: {df['total_risk_score'].median():.2f}")
print(f"  Max: {df['total_risk_score'].max():.2f}")
print(f"  Standard Deviation: {df['total_risk_score'].std():.2f}")

## 6. Alert Generation and Management

Creating and managing alerts for different teams based on risk assessments.

In [ ]:
class AlertManager:
    def __init__(self):
        self.alerts = []
        self.alert_counter = 1
        
    def generate_alert(self, transaction_id, risk_level, risk_score, assigned_team, risk_factors):
        """Generate an alert for high-risk transactions"""
        alert = {
            'alert_id': f"ALERT_{self.alert_counter:06d}",
            'transaction_id': transaction_id,
            'timestamp': datetime.now(),
            'risk_level': risk_level,
            'risk_score': risk_score,
            'assigned_team': assigned_team,
            'risk_factors': risk_factors,
            'status': 'Open',
            'priority': self._determine_priority(risk_level, risk_score)
        }
        self.alerts.append(alert)
        self.alert_counter += 1
        return alert
    
    def _determine_priority(self, risk_level, risk_score):
        """Determine alert priority based on risk level and score"""
        if risk_level == 'High' and risk_score >= 0.8:
            return 'Critical'
        elif risk_level == 'High' and risk_score >= 0.7:
            return 'High'
        elif risk_level == 'Medium':
            return 'Medium'
        else:
            return 'Low'
    
    def get_alerts_by_team(self, team):
        """Get all alerts assigned to a specific team"""
        return [alert for alert in self.alerts if alert['assigned_team'] == team]
    
    def update_alert_status(self, alert_id, new_status, notes=""):
        """Update the status of an alert"""
        for alert in self.alerts:
            if alert['alert_id'] == alert_id:
                alert['status'] = new_status
                alert['last_updated'] = datetime.now()
                if notes:
                    alert['notes'] = notes
                break

# Initialize Alert Manager
alert_manager = AlertManager()

# Generate alerts for high-risk transactions
for _, row in transactions_df.iterrows():
    risk_score = row['risk_score']
    risk_level = row['risk_level']
    
    if risk_level in ['Medium', 'High']:
        risk_factors = []
        if row['amount'] > 50000:
            risk_factors.append('High Amount')
        if row['sender_country'] != row['receiver_country']:
            risk_factors.append('Cross-border')
        if row['previous_suspicious_activity']:
            risk_factors.append('Previous Suspicious Activity')
        if row['transaction_frequency'] > 5:
            risk_factors.append('High Frequency')
        
        alert = alert_manager.generate_alert(
            transaction_id=row['transaction_id'],
            risk_level=risk_level,
            risk_score=risk_score,
            assigned_team=row['assigned_team'],
            risk_factors=risk_factors
        )

print(f"Generated {len(alert_manager.alerts)} alerts")
print(f"Critical alerts: {len([a for a in alert_manager.alerts if a['priority'] == 'Critical'])}")
print(f"High priority alerts: {len([a for a in alert_manager.alerts if a['priority'] == 'High'])}")

# Display sample alerts
alerts_df = pd.DataFrame(alert_manager.alerts)
print("\nSample Alerts:")
print(alerts_df[['alert_id', 'transaction_id', 'risk_level', 'risk_score', 'assigned_team', 'priority']].head())

## 7. Audit Trail and Compliance

Maintaining comprehensive audit trails for regulatory compliance.

In [ ]:
class AuditTrail:
    def __init__(self):
        self.audit_log = []
        
    def log_transaction_analysis(self, transaction_id, risk_score, risk_level, analyst_id="SYSTEM"):
        """Log transaction analysis activity"""
        log_entry = {
            'timestamp': datetime.now(),
            'activity_type': 'TRANSACTION_ANALYSIS',
            'transaction_id': transaction_id,
            'risk_score': risk_score,
            'risk_level': risk_level,
            'analyst_id': analyst_id,
            'details': f"Transaction analyzed with risk score {risk_score:.3f} and level {risk_level}"
        }
        self.audit_log.append(log_entry)
    
    def log_alert_generation(self, alert_id, transaction_id, assigned_team, analyst_id="SYSTEM"):
        """Log alert generation activity"""
        log_entry = {
            'timestamp': datetime.now(),
            'activity_type': 'ALERT_GENERATION',
            'alert_id': alert_id,
            'transaction_id': transaction_id,
            'assigned_team': assigned_team,
            'analyst_id': analyst_id,
            'details': f"Alert {alert_id} generated and assigned to {assigned_team}"
        }
        self.audit_log.append(log_entry)
    
    def log_kyc_check(self, customer_id, kyc_status, analyst_id="SYSTEM"):
        """Log KYC compliance check"""
        log_entry = {
            'timestamp': datetime.now(),
            'activity_type': 'KYC_CHECK',
            'customer_id': customer_id,
            'kyc_status': kyc_status,
            'analyst_id': analyst_id,
            'details': f"KYC check performed for customer {customer_id} with status {kyc_status}"
        }
        self.audit_log.append(log_entry)
    
    def generate_compliance_report(self, start_date=None, end_date=None):
        """Generate compliance report for regulatory submission"""
        if start_date is None:
            start_date = datetime.now() - timedelta(days=30)
        if end_date is None:
            end_date = datetime.now()
        
        filtered_logs = [
            log for log in self.audit_log 
            if start_date <= log['timestamp'] <= end_date
        ]
        
        report = {
            'report_period': f"{start_date.strftime('%Y-%m-%d')} to {end_date.strftime('%Y-%m-%d')}",
            'total_activities': len(filtered_logs),
            'transaction_analyses': len([l for l in filtered_logs if l['activity_type'] == 'TRANSACTION_ANALYSIS']),
            'alerts_generated': len([l for l in filtered_logs if l['activity_type'] == 'ALERT_GENERATION']),
            'kyc_checks': len([l for l in filtered_logs if l['activity_type'] == 'KYC_CHECK']),
            'detailed_logs': filtered_logs
        }
        
        return report

# Initialize Audit Trail
audit_trail = AuditTrail()

# Log activities for processed transactions
for _, row in transactions_df.iterrows():
    # Log transaction analysis
    audit_trail.log_transaction_analysis(
        transaction_id=row['transaction_id'],
        risk_score=row['risk_score'],
        risk_level=row['risk_level']
    )
    
    # Log KYC check
    audit_trail.log_kyc_check(
        customer_id=row['sender_id'],
        kyc_status=row['kyc_status']
    )

# Log alert generation
for alert in alert_manager.alerts:
    audit_trail.log_alert_generation(
        alert_id=alert['alert_id'],
        transaction_id=alert['transaction_id'],
        assigned_team=alert['assigned_team']
    )

# Generate compliance report
compliance_report = audit_trail.generate_compliance_report()
print("Compliance Report Summary:")
print(f"Report Period: {compliance_report['report_period']}")
print(f"Total Activities: {compliance_report['total_activities']}")
print(f"Transaction Analyses: {compliance_report['transaction_analyses']}")
print(f"Alerts Generated: {compliance_report['alerts_generated']}")
print(f"KYC Checks: {compliance_report['kyc_checks']}")

# Display recent audit log entries
audit_df = pd.DataFrame(audit_trail.audit_log)
print("\nRecent Audit Log Entries:")
print(audit_df[['timestamp', 'activity_type', 'details']].tail())

## 8. Data Visualization and Reporting

Comprehensive dashboards and visualizations for AML monitoring insights.

In [ ]:
# Risk Score Distribution Analysis
fig, axes = plt.subplots(2, 2, figsize=(15, 12))

# Risk Score Distribution
axes[0, 0].hist(transactions_df['risk_score'], bins=30, alpha=0.7, color='skyblue', edgecolor='black')
axes[0, 0].set_title('Risk Score Distribution')
axes[0, 0].set_xlabel('Risk Score')
axes[0, 0].set_ylabel('Frequency')
axes[0, 0].axvline(transactions_df['risk_score'].mean(), color='red', linestyle='--', label=f'Mean: {transactions_df["risk_score"].mean():.3f}')
axes[0, 0].legend()

# Risk Level Distribution
risk_level_counts = transactions_df['risk_level'].value_counts()
axes[0, 1].pie(risk_level_counts.values, labels=risk_level_counts.index, autopct='%1.1f%%', 
               colors=['lightcoral', 'lightsalmon', 'lightgreen'])
axes[0, 1].set_title('Risk Level Distribution')

# Transaction Amount vs Risk Score
scatter = axes[1, 0].scatter(transactions_df['amount'], transactions_df['risk_score'], 
                           c=transactions_df['risk_score'], cmap='Reds', alpha=0.6)
axes[1, 0].set_title('Transaction Amount vs Risk Score')
axes[1, 0].set_xlabel('Transaction Amount')
axes[1, 0].set_ylabel('Risk Score')
plt.colorbar(scatter, ax=axes[1, 0])

# Team Assignment Distribution
team_counts = transactions_df['assigned_team'].value_counts()
axes[1, 1].bar(team_counts.index, team_counts.values, color=['lightblue', 'lightcoral', 'lightgreen'])
axes[1, 1].set_title('Team Assignment Distribution')
axes[1, 1].set_xlabel('Team')
axes[1, 1].set_ylabel('Number of Cases')
axes[1, 1].tick_params(axis='x', rotation=45)

plt.tight_layout()
plt.show()

# KYC and EDD Status Analysis
fig, axes = plt.subplots(1, 2, figsize=(12, 5))

# KYC Status Distribution
kyc_counts = transactions_df['kyc_status'].value_counts()
axes[0].pie(kyc_counts.values, labels=kyc_counts.index, autopct='%1.1f%%', 
           colors=['lightgreen', 'lightyellow', 'lightcoral'])
axes[0].set_title('KYC Status Distribution')

# EDD Required Distribution
edd_counts = transactions_df['edd_required'].value_counts()
colors = ['lightblue', 'lightpink']
axes[1].bar(['Not Required', 'Required'], edd_counts.values, color=colors)
axes[1].set_title('EDD Requirement Distribution')
axes[1].set_ylabel('Number of Transactions')

plt.tight_layout()
plt.show()

# Alert Priority and Status Analysis
if alert_manager.alerts:
    alerts_df = pd.DataFrame(alert_manager.alerts)
    
    fig, axes = plt.subplots(1, 2, figsize=(12, 5))
    
    # Alert Priority Distribution
    priority_counts = alerts_df['priority'].value_counts()
    colors = {'Critical': 'red', 'High': 'orange', 'Medium': 'yellow', 'Low': 'green'}
    alert_colors = [colors.get(priority, 'gray') for priority in priority_counts.index]
    axes[0].bar(priority_counts.index, priority_counts.values, color=alert_colors)
    axes[0].set_title('Alert Priority Distribution')
    axes[0].set_ylabel('Number of Alerts')
    
    # Risk Score by Team
    team_risk_scores = transactions_df.groupby('assigned_team')['risk_score'].mean()
    axes[1].bar(team_risk_scores.index, team_risk_scores.values, color=['lightblue', 'lightcoral', 'lightgreen'])
    axes[1].set_title('Average Risk Score by Team')
    axes[1].set_ylabel('Average Risk Score')
    axes[1].tick_params(axis='x', rotation=45)
    
    plt.tight_layout()
    plt.show()

# Summary Statistics
print("=== AML MONITORING SUMMARY REPORT ===")
print(f"Total Transactions Processed: {len(transactions_df)}")
print(f"Average Risk Score: {transactions_df['risk_score'].mean():.3f}")
print(f"High Risk Transactions: {len(transactions_df[transactions_df['risk_level'] == 'High'])}")
print(f"Medium Risk Transactions: {len(transactions_df[transactions_df['risk_level'] == 'Medium'])}")
print(f"Low Risk Transactions: {len(transactions_df[transactions_df['risk_level'] == 'Low'])}")
print(f"Total Alerts Generated: {len(alert_manager.alerts)}")
print(f"KYC Compliant: {len(transactions_df[transactions_df['kyc_status'] == 'Compliant'])}")
print(f"EDD Required: {len(transactions_df[transactions_df['edd_required'] == True])}")
print(f"Cross-border Transactions: {len(transactions_df[transactions_df['sender_country'] != transactions_df['receiver_country']])}")

# Team Performance Metrics
print("\n=== TEAM PERFORMANCE METRICS ===")
team_metrics = transactions_df.groupby('assigned_team').agg({
    'risk_score': ['mean', 'max', 'count'],
    'amount': ['sum', 'mean']
}).round(3)
print(team_metrics)

## 9. Conclusion

This AML monitoring system provides comprehensive real-time analysis of financial transactions with:

- **Advanced Risk Scoring**: Multi-factor risk assessment combining transaction patterns, amounts, and customer profiles
- **Intelligent Team Assignment**: Automated routing of cases to appropriate specialist teams
- **KYC/EDD Compliance**: Integrated compliance monitoring and enhanced due diligence workflows
- **Alert Management**: Prioritized alert system with comprehensive tracking and management
- **Audit Trail**: Complete regulatory compliance with detailed activity logging
- **Visualization Dashboard**: Real-time insights and performance metrics

The system successfully processes transactions, identifies potential money laundering activities, and ensures regulatory compliance while providing actionable insights for financial crime prevention teams.